In [ ]:
# 1a) ????

import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 200

In [ ]:
def gaussdx(size, gamma):
    if size > 2 * 3 * gamma + 1:
        size = 2 * 3 * gamma + 1
    gaus = np.arange(-size,size+1)
    gaus = gaus.astype(float)
    gaus[:] = np.multiply( -1/(np.sqrt(2 * np.pi) * np.power(gamma,3)), np.multiply(gaus[:],np.exp(np.divide(-np.square(gaus[:]),2*np.square(gamma)))) )
    return gaus / sum(abs(gaus))

def gauss(size, gamma):
    if size > 2 * 3 * gamma + 1:
        size = 2 * 3 * gamma + 1
    gaus = np.arange(-size,size+1)
    gaus = gaus.astype(float)
    gaus[:] = np.multiply(1/(np.sqrt(2 * np.pi) * gamma), np.exp(np.divide(-np.square(gaus[:]),2*np.square(gamma))) )
    return gaus / sum(gaus)

In [ ]:
def c(size):
    impulse = np.zeros((size,size))
    impulse[int(size/2),int(size/2)]=255
    g = (np.array(gauss(8,2),ndmin=2))
    d = (np.array(gaussdx(8,2),ndmin=2))

    plt.subplot(2,3,1)
    plt.imshow( impulse ,cmap="gray")
    plt.title("impulse")
    plt.subplot(2,3,2)
    plt.title("G, Dt")
    plt.imshow( cv2.filter2D(cv2.filter2D(impulse,-1,g,borderType=4),-1,cv2.transpose(np.flip(d)),borderType=4) ,cmap="gray")
    plt.subplot(2,3,3)
    plt.title("D, Gt")
    plt.imshow( cv2.filter2D(cv2.filter2D(impulse,-1, np.flip(d) ,borderType=4),-1,cv2.transpose(g),borderType=4) ,cmap="gray")
    plt.subplot(2,3,4)
    plt.title("G, Gt")
    plt.imshow( cv2.filter2D(cv2.filter2D(impulse,-1,g,borderType=4),-1,cv2.transpose(g),borderType=4) ,cmap="gray")
    plt.subplot(2,3,5)
    plt.imshow( cv2.filter2D(cv2.filter2D(impulse,-1, cv2.transpose(g) ,borderType=4),-1,(np.flip(d)),borderType=4) ,cmap="gray")
    plt.title("Gt, D")
    plt.subplot(2,3,6)
    plt.imshow( cv2.filter2D(cv2.filter2D(impulse,-1, cv2.transpose(np.flip(d)) ,borderType=4),-1,g,borderType=4) ,cmap="gray")
    plt.title("Dt, G")

    plt.tight_layout()
    plt.show()

c(25)

In [ ]:
def partial_derivatives(image,size,sigma):
    image = image.astype(float)
    g = gauss(size,sigma)
    d = gaussdx(size,sigma)
    ix = cv2.filter2D(cv2.filter2D(image,-1, g ,borderType=4),-1,cv2.transpose(np.flip(d)),borderType=4) 
    iy = cv2.filter2D(cv2.filter2D(image,-1, np.flip(d) ,borderType=4),-1,cv2.transpose(g),borderType=4)
    ixx = cv2.filter2D(cv2.filter2D(ix,-1, g ,borderType=4),-1,cv2.transpose(np.flip(d)),borderType=4) 
    ixy = cv2.filter2D(cv2.filter2D(ix,-1, np.flip(d) ,borderType=4),-1,cv2.transpose(g),borderType=4) 
    iyy = cv2.filter2D(cv2.filter2D(iy,-1, np.flip(d) ,borderType=4),-1,cv2.transpose(g),borderType=4) 
    return [ix,iy,ixx,ixy,iyy]
# size was 3

def gradient_magnitute(image,size,sigma):
    [a,b,_,_,_] = partial_derivatives(image,size,sigma)
    image = image.astype(float)

    magnitude = np.sqrt(np.add(np.square(a),np.square(b)))
    angle = np.arctan2( b,a )
    return [magnitude, angle]


i = cv2.imread("./images/museum.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

[ix,iy,ixx,ixy,iyy] = partial_derivatives(i,3,0.5)
[mag,ang] = gradient_magnitute(i,3,0.5)

plt.subplot(2,4,1)
plt.imshow( i ,cmap="gray")
plt.subplot(2,4,2)
plt.imshow( ix ,cmap="gray")
plt.subplot(2,4,3)
plt.imshow( iy ,cmap="gray")
plt.subplot(2,4,4)
plt.imshow( ixx ,cmap="gray")
plt.subplot(2,4,5)
plt.imshow( ixy ,cmap="gray")
plt.subplot(2,4,6)
plt.imshow( iyy ,cmap="gray")
plt.subplot(2,4,7)
plt.imshow( mag ,cmap="gray")
plt.subplot(2,4,8)
plt.imshow( ang ,cmap="gray")
plt.show()

    

In [ ]:
def get_angle_bin(angle):
    angle = (np.around((angle/np.pi)*180,decimals=3))

    if angle > 0:
        angle -= 0.0001
    elif angle < 0:
        angle += 0.0001
    angle = int((angle)/22.5)

    if angle == 0:  
        return 0
    elif angle == 2 or angle == 1:
        return 1
    elif angle == 3 or angle == 4:
        return 2
    elif angle == 5 or angle == 6:
        return 3
    elif angle == 7 or angle == -7:
        return 4
    elif angle == -5 or angle == -6:
        return 5
    elif angle == -4 or angle == -3:
        return 6
    elif angle == -2 or angle == -1:
        return 7


def myhist3(image):
    l = 128,16
    mag,angle = gradient_magnitute(image,3,0.5)
    histogrami = np.zeros((8,8,8))
    for i in range(8):
        for j in range(8):
            for x in range(i*16,i*16+16):
                for y in range(j*16,j*16+16):
                    histogrami[i,j, get_angle_bin(angle[x,y])] += mag[x,y]

    return(histogrami.reshape(-1))

def compare_histogram(hist1,hist2,measure_type):
    if measure_type == "l2":
        return np.sqrt(np.sum(np.square(np.subtract(hist1,hist2)))) 
    if measure_type == "chi":
        return 1/2 * np.sum(np.divide( np.square(np.subtract(hist1,hist2)), np.add(np.add(hist1,hist2),1e-10 )))
    if measure_type == "intersection":
        return 1 - np.sum(np.minimum(hist1,hist2))
    if measure_type == "hellinger":
        return np.sqrt( np.multiply( 0.5 ,np.sum(np.square(np.subtract(np.sqrt(hist1), np.sqrt(hist2))))))  


def get_all(path):

    histograms = np.zeros((120,8*8*8))
    names = []
    index = 0
    for name in os.listdir(path):
        i = cv2.imread(path + name)
        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
        i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

        histograms[index] = np.copy(myhist3(i))
        names.append(name)
        index += 1
    return [histograms,names]

[histograms,names] = get_all("./dataset/")

def omgwow():
        
    main = 19
    tip = "hellinger"
    w = 10

    rankings = np.reshape(np.zeros(120 * 2), (120,2))
    for indx, hist in enumerate(histograms):
        rankings[indx,0] = indx
        rankings[indx,1] = compare_histogram( histograms[main], hist, tip)

    gr = np.copy(rankings)
    rankings = rankings[rankings[:,1].argsort()]

    i = cv2.imread("./dataset/" + names[int(rankings[0,0])])
    i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
    i2 = cv2.imread("./dataset/" + names[int(rankings[1,0])])
    i2 = cv2.cvtColor(i2, cv2.COLOR_BGR2RGB)
    i3 = cv2.imread("./dataset/" + names[int(rankings[2,0])])
    i3 = cv2.cvtColor(i3, cv2.COLOR_BGR2RGB)
    i4 = cv2.imread("./dataset/" + names[int(rankings[3,0])])
    i4 = cv2.cvtColor(i4, cv2.COLOR_BGR2RGB)
    i5 = cv2.imread("./dataset/" + names[int(rankings[4,0])])
    i5 = cv2.cvtColor(i5, cv2.COLOR_BGR2RGB)
    i6 = cv2.imread("./dataset/" + names[int(rankings[5,0])])
    i6 = cv2.cvtColor(i6, cv2.COLOR_BGR2RGB)

    plt.subplot(2,6,1)
    plt.title("slika " + str(int(rankings[0,0])+1))
    plt.imshow(i)
    plt.subplot(2,6,7)
    plt.title(tip + " = " + str(round(rankings[0,1],3)))
    plt.bar(np.arange(len(histograms[0])), histograms[int(rankings[0,0])], width=w)
    plt.tight_layout()

    plt.subplot(2,6,2)
    plt.title("slika " + str(int(rankings[1,0])+1))
    plt.imshow(i2)
    plt.subplot(2,6,8)
    plt.title(tip + " = " + str(round(rankings[1,1],3)))
    plt.bar(np.arange(len(histograms[0])), histograms[int(rankings[1,0])], width=w)
    plt.tight_layout()

    plt.subplot(2,6,3)
    plt.title("slika " + str(int(rankings[2,0])+1))
    plt.imshow(i3)
    plt.subplot(2,6,9)
    plt.title(tip + " = " + str(round(rankings[2,1],3)))
    plt.bar(np.arange(len(histograms[0])), histograms[int(rankings[2,0])], width=w)
    plt.tight_layout()

    plt.subplot(2,6,4)
    plt.title("slika " + str(int(rankings[3,0])+1))
    plt.imshow(i4)
    plt.subplot(2,6,10)
    plt.title(tip + " = " + str(round(rankings[3,1],3)))
    plt.bar(np.arange(len(histograms[0])), histograms[int(rankings[3,0])], width=w)
    plt.tight_layout()

    plt.subplot(2,6,5)
    plt.title("slika " + str(int(rankings[4,0])+1))
    plt.imshow(i5)
    plt.subplot(2,6,11)
    plt.title(tip + " = " + str(round(rankings[4,1],3)))
    plt.bar(np.arange(len(histograms[0])), histograms[int(rankings[4,0])], width=w)
    plt.tight_layout()

    plt.subplot(2,6,6)
    plt.title("slika " + str(int(rankings[5,0])+1))
    plt.imshow(i6)
    plt.subplot(2,6,12)
    plt.title(tip + " = " + str(round(rankings[5,1],3)))
    plt.bar(np.arange(len(histograms[0])), histograms[int(rankings[5,0])], width=w)
    plt.tight_layout()

    plt.show()

omgwow()

In [ ]:
def findedges(image,size,sigma,theta):
    [mag,ang] = gradient_magnitute(image,size,sigma)
    for x in range(len(mag)):
        for y in range(len(mag[0])):
            if mag[x,y] >= theta:
                mag[x,y] = 1
            else:
                mag[x,y] = 0
    return mag


i = cv2.imread("./images/museum.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

rows = 1
cols = 4
plt.subplot(rows,cols,1)
plt.imshow( findedges(i,3,1,20), cmap = "gray" )
plt.subplot(rows,cols,2)
plt.imshow( findedges(i,3,1,38) ,cmap="gray")
plt.subplot(rows,cols,3)
plt.imshow( findedges(i,3,1,44) ,cmap="gray")
plt.subplot(rows,cols,4)
plt.imshow( findedges(i,3,1,60) ,cmap="gray")
plt.show()

In [ ]:
def getdirection(angle):
    angle = (np.around((angle/np.pi)*180,decimals=3))

    if angle > 0:
        angle -= 0.0001
    elif angle < 0:
        angle += 0.0001
    angle = int((angle)/22.5)
    if angle == 0 or angle == 7 or angle == -7:  #levodenso
        return np.array([[0,1],[0,-1]])
    if angle == -1 or angle == -2 or angle == 6 or angle == 5: #levogor desnodol
        return np.array([[-1,1],[1,-1]])
    if angle == -3 or angle == -4 or angle == 4 or angle == 3: #gordol
        return np.array([[1,0],[-1,0]]) 
    if angle == -5 or angle == -6 or angle == 2 or angle == 1: #levodol densnogor
        return np.array([[-1,-1],[1,1]])



def findedgesmaxima(image,size,sigma,theta):
    [mag,ang] = gradient_magnitute(image,size,sigma)

    mag[ mag < theta ] = 0

    mag = np.pad(mag, [(1,1),(1,1)], mode="constant")

    for x in range( 1, len(mag)-1 ):
        for y in range( 1, len(mag[0])-1 ):
            if mag[x,y] > 0:
                dirs = getdirection(ang[x-1,y-1])
                if mag[x,y] <= mag[ x + dirs[0,0], y + dirs[0,1] ] or mag[x,y] <= mag[ x + dirs[1,0], y + dirs[1,1] ]:
                    mag[x,y] = 0

    mag = mag[1:len(mag)-1, 1:len(mag[0])-1]


    # mag = hysterisis(mag,theta,theta+25)
    original = np.copy(mag)
    mag[ mag > 0 ] = 1
    return original,mag

def hysterisis(mag,low,high):
    mag = np.copy(mag)
    mag[ mag < low ] = 0

    magg = np.copy(mag)
    magg[ magg >= low ] = 255
    magg = np.array(magg, dtype=np.uint8)
    a,b,c,d = cv2.connectedComponentsWithStats(magg)

    for i in range(a):
        x,y = np.nonzero( b == i )
        if(np.amax(mag[x,y]) <= high):
            mag[x,y] = 0
    mag[mag > 0] = 1
    return mag

i = cv2.imread("./images/museum.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

rows = 2
cols = 2
a = findedges(i,3,1,25)
orig,m = findedgesmaxima(i,3,1,25)

origg,mm = findedgesmaxima(i,3,1,5)


# origg = np.copy(origg)

orig = hysterisis(origg,10,40)

origg = hysterisis(origg,25,40)

plt.subplot(rows,cols,1)
plt.imshow(a, cmap = "gray")

plt.subplot(rows,cols,2)
plt.imshow(m, cmap="gray")
plt.subplot(rows,cols,3)
plt.imshow(orig, cmap="gray")
plt.subplot(rows,cols,4)
plt.imshow(origg, cmap="gray")
plt.show()



In [ ]:
p = [10,10]
theta = np.arange(300)
distance = np.zeros(300)
distance[:] = np.add(np.multiply(p[0],np.cos(theta[:]/180*np.pi)),np.multiply(p[1],np.sin(theta[:]/180*np.pi)))

plt.subplot(2,2,1)
plt.ylim([-150,150])
plt.xlim([0,300])
plt.plot(distance)

p = [30,60]
distance[:] = np.add(np.multiply(p[0],np.cos(theta[:]/180*np.pi)),np.multiply(p[1],np.sin(theta[:]/180*np.pi)))
plt.subplot(2,2,2)
plt.ylim([-150,150])
plt.xlim([0,300])
plt.plot(distance)

p = [50,20]
distance[:] = np.add(np.multiply(p[0],np.cos(theta[:]/180*np.pi)),np.multiply(p[1],np.sin(theta[:]/180*np.pi)))
plt.subplot(2,2,3)
plt.ylim([-150,150])
plt.xlim([0,300])
plt.plot(distance)

p = [80,90]
distance[:] = np.add(np.multiply(p[0],np.cos(theta[:]/180*np.pi)),np.multiply(p[1],np.sin(theta[:]/180*np.pi)))
plt.subplot(2,2,4)
plt.ylim([-150,150])
plt.xlim([0,300])
plt.plot(distance)
plt.show()

In [ ]:
def hough_find_lines(bi,thetabins,rhobins):
    A = np.zeros([thetabins,rhobins])
    d = int(np.sqrt( np.square(len(bi)) + np.square(len(bi[0])) ))
    leftSpan = 2*d
    rightSpan = rhobins

    theta = np.arange(-thetabins/2,thetabins/2)
    theta = theta / (thetabins/2) * np.pi/2
    rho = np.arange(-d,d+1,2*d/rhobins)
    
    for x in range( len(bi) ):
        for y in range( len(bi[0]) ):
            if bi[x,y] == 1:
                distance = np.zeros(rhobins)
                distance[:] = np.add(np.multiply(x,np.cos(theta[:])),np.multiply(y,np.sin(theta[:])))
                distance[:] = np.multiply(np.divide(np.add(distance[:] , d) , float(leftSpan)) , rightSpan)
                for i,dist in enumerate(distance):
                    A[int(dist),i] += 1

    return [A,theta,rho]

i = cv2.imread("./images/synthetic.png")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

edges = findedges(i,3,1,38)
[hough,_,_] = hough_find_lines(edges, 300,300)

rows = 1
cols = 3
plt.subplot(rows,cols,1)
plt.imshow( hough,cmap="jet" )

plt.subplot(rows,cols,2)
i = cv2.imread("./images/oneline.png")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)
edges = findedges(i,3,1,38)
[hough,_,_] = hough_find_lines(edges, 300,300)
plt.imshow( hough, cmap="jet" )

plt.subplot(rows,cols,3)
i = cv2.imread("./images/rectangle.png")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)
edges = findedges(i,3,1,38)
[hough,_,_] = hough_find_lines(edges, 300,300)
plt.imshow( hough,cmap="jet" )
plt.show( )

In [ ]:
def nonmaxima_suppression_box(h):
    h = np.pad(h, [(1,1),(1,1)], mode="constant")
    for x in range( 1,len(h)-1 ):
        for y in range( 1,len(h[0])-1 ):
            main = h[x,y]
            for i in range(x-1,x+2):
                for j in range(y-1,y+2):
                    if h[i,j] >= main and not (i == x and j == y):
                        h[x,y] = 0
                        break

    return h[1:len(h)-1, 1:len(h)-1]

# nonmaxima_suppression_box(1)

In [ ]:
def draw_line(rho, theta, max_rho):
    a = np.cos(theta)
    b = np.sin(theta)

    x0 = a*rho
    y0 = b*rho

    x1 = int(x0 + max_rho*(-b))
    y1 = int(y0 + max_rho*(a))
    x2 = int(x0 - max_rho*(-b))
    y2 = int(y0 - max_rho*(a))
    plt.plot((y1,y2),(x1,x2),"#3dff7b",linewidth=0.7)


def getthresholded(h,thetas,rhos,thresh):
    h[ h < thresh ] = 0
    [x,y] = np.nonzero(h)
    for i in range( len(x) ):
        draw_line( rhos[ x[i] ], thetas[ y[i] ], -rhos[0] )
     

hs = 300

i = cv2.imread("./images/synthetic.png")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)
o,edges = findedgesmaxima(i,3,1,10)
[hough,t,r] = hough_find_lines(edges, hs,hs)
rows = 2
cols = 3
plt.subplot(rows,cols,1)
plt.imshow( hough,cmap="jet" )

hough = nonmaxima_suppression_box(hough)
plt.subplot(rows,cols,4)
getthresholded(hough,t,r,20)
plt.imshow( i,cmap="jet" )



plt.subplot(rows,cols,2)
i = cv2.imread("./images/oneline.png")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)
o,edges = findedgesmaxima(i,3,1,5)
[hough,t,r] = hough_find_lines(edges, hs,hs)
plt.imshow( hough, cmap="jet" )

hough = nonmaxima_suppression_box(hough)
plt.subplot(rows,cols,5)
getthresholded(hough,t,r,300)
plt.imshow( i,cmap="jet" )




plt.subplot(rows,cols,3)
i = cv2.imread("./images/rectangle.png")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)
o,edges = findedgesmaxima(i,3,1,5)
[hough,t,r] = hough_find_lines(edges, hs,hs)
plt.imshow( hough,cmap="jet" )

hough = nonmaxima_suppression_box(hough)
plt.subplot(rows,cols,6)
getthresholded(hough,t,r,100)
plt.imshow( i,cmap="jet" )


plt.show( )

In [ ]:
rows = 2
cols = 2
hs = 175
def gettop10(h,thetas,rhos):
    rank = np.zeros( (len(h)*len(h[0]),3) )
    i = 0

    for x in range(len(h)):
        for y in range( len(h[0])):
            rank[i][0] = x         
            rank[i][1] = y         
            rank[i][2] = h[x,y]
            i += 1

    rank[:,2] = -rank[:,2]
    rank = rank[rank[:, 2].argsort()]
    rank[:,2] = -rank[:,2]

    for i in range( 10 ):
        draw_line( rhos[ int(rank[i,0]) ], thetas[ int(rank[i,1]) ], -rhos[0] )
     

plt.subplot(rows,cols,1)
i = cv2.imread("./images/bricks.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
ii = np.copy(i)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)
o,edges = findedgesmaxima(i,10,3,10)
[hough,t,r] = hough_find_lines(edges, hs,hs)
hough = nonmaxima_suppression_box(hough)
plt.imshow( hough,cmap="jet" )

plt.subplot(rows,cols,2)
gettop10(hough,t,r)
plt.imshow( ii )


plt.subplot(rows,cols,3)
j = cv2.imread("./images/pier.jpg")
j = cv2.cvtColor(j, cv2.COLOR_BGR2RGB)
jj = np.copy(j)
j = cv2.cvtColor(j, cv2.COLOR_RGB2GRAY)



o,edges = findedgesmaxima(j,15,1.5,5) #7
o = hysterisis(o,5,15)
                            #edges
[hough,t,r] = hough_find_lines(o, hs,hs)
hough = nonmaxima_suppression_box(hough)

plt.imshow( hough,cmap="jet" )



plt.subplot(rows,cols,4)
gettop10(hough,t,r)
plt.imshow( jj )
plt.show()
